<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/space_travel_demo_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colab-env --upgrade -q
!pip install openai -q
import os
import colab_env
import json
import openai

In [10]:
from openai import OpenAI
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [13]:
openai.api_key = "OPENAI_API_KEY"  # Replace with your actual API key

In [14]:
openai.api_key

'OPENAI_API_KEY'

In [ ]:
from openai import OpenAI
client = OpenAI()

modellist=client.models.list()
modellist.data

## EARTH TO MARS

In [51]:
import os
import openai
from typing import List, Dict

# Define a function to interact with the LLM
def generate_flight_plan_with_llm(prompt: str, system_prompt: str = None, model: str = "gpt-4", temperature: float = 0.7) -> str:
    # Set your OpenAI API key using os.getenv
    openai.api_key = os.getenv("OPENAI_API_KEY")

    messages = [{"role": "user", "content": prompt}]
    if system_prompt:
        messages.insert(0, {"role": "system", "content": system_prompt})

    # Generate flight plan with the specified LLM using chat completions
    response = openai.chat.completions.create(
        model=model,  # Specify the LLM model to use
        messages=messages,
        max_completion_tokens=2048,  # Increased max_completion_tokens to 2048
        n=1,  # Generate one response
        stop=None,  # Explicitly set stop to None
        temperature=temperature,  # Adjust for creativity
    )

    # Extract the flight plan from the response
    flight_plan = response.choices[0].message.content.strip()

    return flight_plan


# Define a class for the AI agent
class SpaceFlightPlanningAgent:
    def __init__(self, llm_models: List[str]):
        self.llm_models = llm_models

    def generate_flight_plan(
            self,
            origin: str,
            destination: str,
            spacecraft: str,
            mission_objectives: List[str],
            constraints: List[str],
    ) -> Dict[str, Dict[str, str]]:
        system_prompt = f"""
        You are a helpful and informative space flight planner.

        You are tasked with generating a detailed flight plan for a mission from {origin} to {destination}
        using a {spacecraft}. The mission objectives are: {mission_objectives}. The flight plan
        must satisfy the following constraints: {constraints}.

        Consider these factors:
        - Orbital mechanics (including Hohmann transfer orbits, delta-v calculations, and
          apogee/perigee)
        - Spacecraft capabilities (delta-v, thrust-to-weight ratio, specific impulse)
        - Gravitational forces of relevant celestial bodies (Sun, Earth, Mars, etc.)
        - Radiation exposure (minimize exposure by considering solar activity and trajectory)
        - Communication windows (ensure contact with Earth throughout the mission)
        - Launch opportunities (identify optimal launch windows based on planetary alignment)
        """

        flight_plan_sections = {
            "Launch Date and Time": """Provide a **specific** launch date and time in UTC, considering optimal launch windows and radiation exposure.
                                 Do not provide a range of dates or suggest future windows. Choose a date and time within the next 5 years.""",  # Modified prompt
            "Trajectory": {  # Trajectory section with iterative prompting strategy
                "initial_prompt": "Provide a summary of the key phases and maneuvers of the trajectory.",
                "follow_up_prompts": [
                    "Describe the launch process, including the type of launch vehicle, the target orbit, and any relevant maneuvers. Provide details about the initial orbit, such as its altitude, inclination, and purpose.",
                    "Explain the Hohmann transfer maneuver, including the delta-v requirements, the timing of the maneuver, and the resulting trajectory.",
                    "Detail the interplanetary cruise phase, including the duration, the communication strategy, and any planned course corrections.",
                    "Describe the approach to Mars and the Mars orbit insertion maneuver, including the delta-v requirements, the target orbit parameters, and any challenges or considerations."
                ]
            },
            "Maneuver Schedule": "Detail each maneuver with its purpose, delta-v requirement, and timing. Include maneuvers like trans-Mars injection, Mars orbit insertion, and any course correction burns.",
            "Communication Plan": "Explain how communication with Earth will be maintained throughout the mission, including the use of the Deep Space Network and any considerations for communication delays or blackouts.",
            "Contingency Plans": "Describe contingency plans for various scenarios, such as launch vehicle failure, missed maneuver opportunities, or spacecraft malfunctions. Address crew safety and mission recovery options."
        }

        all_flight_plans = {}  # Store flight plans for all models

        for model in self.llm_models:
            flight_plan = {}  # Store flight plan sections for current model
            for section_name, section_instructions in flight_plan_sections.items():
                if section_name == "Trajectory":  # Handle Trajectory with iterative prompting
                    # Initial prompt
                    prompt = section_instructions["initial_prompt"]
                    trajectory_details = [generate_flight_plan_with_llm(prompt, system_prompt, model, temperature=0.7)]

                    # Follow-up prompts
                    for sub_question in section_instructions["follow_up_prompts"]:
                        prompt = f"""
                        Specifically, focus on the {section_name}:

                        {sub_question}
                        """
                        trajectory_details.append(generate_flight_plan_with_llm(prompt, system_prompt, model, temperature=0.7))

                    flight_plan[section_name] = "\n".join(trajectory_details)  # Combine responses

                else:
                    prompt = f"""
                    Specifically, focus on the {section_name}:

                    {section_instructions}
                    """
                    flight_plan[section_name] = generate_flight_plan_with_llm(prompt, system_prompt, model, temperature=0.7) # Added temperature parameter
                all_flight_plans[model] = flight_plan  # Add model's flight plan to the dictionary

        return all_flight_plans


# Example usage
if __name__ == "__main__":
    agent = SpaceFlightPlanningAgent(["gpt-4"])  # You can add more models here

    all_flight_plans = agent.generate_flight_plan(
        origin="Earth",
        destination="Mars",
        spacecraft="Orion spacecraft",
        mission_objectives=["Land on Mars", "Collect soil samples"],
        constraints=["Minimize radiation exposure", "Ensure crew safety"]
    )

    # Print the flight plans generated by each LLM
    for model, flight_plan in all_flight_plans.items():
        print(f"Flight plan generated by {model}:")
        for section_name, section_content in flight_plan.items():
            print(f"{section_name}:\n{section_content}\n")

Flight plan generated by gpt-4:
Launch Date and Time:
The optimal launch window for a mission to Mars occurs approximately every 26 months, due to the alignment of Earth and Mars. Within the next 5 years, the two most suitable launch windows would be in late 2022 and late 2024. 

However, considering the requirement to minimize radiation exposure, it would be beneficial to launch during a period of solar minimum, when solar radiation is at its lowest. The sun's activity operates on an approximately 11-year cycle, and the next solar minimum is predicted to occur around 2025.

Taking these factors into account, the most suitable specific date and time for launch would be **December 1, 2024 at 14:00 UTC**.

This date aligns with the optimal launch window and is close to the predicted solar minimum, minimizing radiation exposure. It also allows ample time for mission planning and preparation. This date and time should provide the most efficient trajectory to Mars while ensuring crew safety

## EARTH TO MOON

In [49]:
import os
import openai
from typing import List, Dict
from datetime import datetime, timedelta


def generate_flight_plan_with_llm(prompt: str, system_prompt: str = None, model: str = "gpt-4", temperature: float = 0.7, max_tokens=500) -> str:  # Reduced max_tokens for chunking
    """Generates a response from the LLM."""

    openai.api_key = os.getenv("OPENAI_API_KEY")

    messages = [{"role": "user", "content": prompt}]
    if system_prompt:
        messages.insert(0, {"role": "system", "content": system_prompt})

    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,  # Reduced max_tokens
        n=1,
        stop=None,
        temperature=temperature
    )

    flight_plan = response.choices[0].message.content.strip()
    return flight_plan


def generate_flight_plan_iteratively(prompt: str, system_prompt: str = None, model: str = "gpt-4", temperature: float = 0.7, max_tokens_per_iteration=500) -> str:
    """Generates a flight plan iteratively, handling potential truncations with a response chunking mechanism."""

    full_response = ""
    current_prompt = prompt

    # Chunking logic using 'finish_reason'
    while True:
        response = openai.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": current_prompt}],
            max_tokens=max_tokens_per_iteration,
            n=1,
            stop=None,
            temperature=temperature
        )
        chunk = response.choices[0].message.content.strip()
        full_response += chunk

        # If the model has more to say, continue
        if response.choices[0].finish_reason == "stop":
            break

        # Otherwise, prepare the next prompt and continue
        current_prompt = f"{chunk} Continue:"  # Include the previous chunk in the next prompt

    return full_response


# Define a class for the AI agent
class SpaceFlightPlanningAgent:
    def __init__(self, llm_models: List[str]):
        self.llm_models = llm_models

    def generate_flight_plan(
            self,
            origin: str,
            destination: str,
            spacecraft: str,
            mission_objectives: List[str],
            constraints: List[str],
    ) -> Dict[str, Dict[str, str]]:
        system_prompt = f"""
        You are a helpful and informative space flight planner.

        You are tasked with generating a detailed flight plan for a mission from {origin} to {destination}
        using a {spacecraft}. The mission objectives are: {mission_objectives}. The flight plan
        must satisfy the following constraints: {constraints}.

        Consider these factors:
        - Orbital mechanics (including translunar injection, lunar orbit insertion, and descent/ascent maneuvers)
        - Spacecraft capabilities (delta-v, thrust-to-weight ratio, specific impulse)
        - Gravitational forces of relevant celestial bodies (Earth, Moon)
        - Radiation exposure (minimize exposure by considering solar activity and trajectory)
        - Communication windows (ensure contact with Earth throughout the mission)
        - Launch opportunities (identify optimal launch windows based on lunar phases and Earth-Moon alignment)
        """

        flight_plan_sections = {
            "Launch Date and Time": f"""Provide a **specific** launch date and time in UTC, considering optimal launch windows and radiation exposure.
                                 Do not provide a range of dates or suggest future windows. Choose a date and time within the next 5 years,
                                 starting from {datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S UTC')}. """,  # Modified prompt to include current date and time
            # Trajectory broken down into subsections:
            "Trajectory": {
                "Launch": "Describe the launch process, including the launch vehicle, launch site, and initial parking orbit.",
                "Earth Departure and Translunar Injection": "Describe the Earth departure and translunar injection maneuvers, including the burn durations, delta-v requirements, and resulting trajectory.",
                "Lunar Orbit Insertion": "Describe the lunar orbit insertion maneuver, including the burn duration, delta-v requirements, and target lunar orbit.",
                "Lunar Descent and Landing": "Describe the lunar descent and landing process, including the landing site selection and landing procedures.",
                "Lunar Surface Operations": "Describe the planned activities on the lunar surface, including scientific experiments and sample collection.",
                "Lunar Ascent and Return": "Describe the lunar ascent and return trajectory, including the maneuvers and burn durations.",
                "Earth Re-entry and Landing": "Describe the Earth re-entry and landing process, including the atmospheric entry and splashdown or landing procedures."
            },
            "Maneuver Schedule": "Detail each maneuver with its purpose, delta-v requirement, and timing. Include maneuvers like translunar injection, lunar orbit insertion, descent, ascent, and any course correction burns.",
            "Communication Plan": "Explain how communication with Earth will be maintained throughout the mission, including the use of ground stations and any considerations for communication delays or blackouts.",
            "Contingency Plans": "Describe contingency plans for various scenarios, such as launch vehicle failure, missed maneuver opportunities, or spacecraft malfunctions. Address crew safety and mission recovery options."
        }

        all_flight_plans = {}  # Store flight plans for all models

        for model in self.llm_models:
            flight_plan = {}  # Store flight plan sections for current model
            for section_name, section_instructions in flight_plan_sections.items():

                # Handle Trajectory subsections:
                if section_name == "Trajectory":
                    trajectory_details = ""
                    for subsection_name, subsection_instructions in section_instructions.items():
                        prompt = f"""
                        Specifically, focus on the {subsection_name} of the Trajectory:

                        {subsection_instructions}
                        """
                        response = generate_flight_plan_iteratively(prompt, system_prompt, model, temperature=0.7)
                        trajectory_details += f"\n**{subsection_name}:**\n{response}\n"
                    flight_plan[section_name] = trajectory_details  # Store the combined trajectory details
                else:
                    # Your existing code for other sections remains the same
                    prompt = f"""
                    Specifically, focus on the {section_name}:

                    {section_instructions}
                    """
                    response = generate_flight_plan_with_llm(prompt, system_prompt, model, temperature=0.7)

                    # Response Evaluation and Adjustment (if needed):
                    retry_count = 0
                    temp = 0.7  # Initial temperature
                    while ("know" in response or "spend several months in this phase" in response or len(response) < 100) and retry_count < 3:
                        print(f"Response for {section_name} was truncated or incomplete. Retrying with temperature {temp}...")
                        # Retry with the same prompt, but use the original function to avoid infinite loop
                        response = generate_flight_plan_with_llm(prompt, system_prompt, model, temperature=temp)
                        retry_count += 1
                        temp += 0.1  # Increase temperature for next retry

                    flight_plan[section_name] = response  # Store the final (hopefully complete) response

                all_flight_plans[model] = flight_plan  # Add model's flight plan to the dictionary # Moved outside the inner loop

        return all_flight_plans

# Example usage
if __name__ == "__main__":
    agent = SpaceFlightPlanningAgent(["gpt-4"])  # You can add more models here

    all_flight_plans = agent.generate_flight_plan(
        origin="Earth",
        destination="Moon",
        spacecraft="Orion spacecraft",
        mission_objectives=["Land on the Moon", "Conduct scientific experiments"],
        constraints=["Minimize radiation exposure", "Ensure crew safety"]
    )

    # Print the flight plans generated by each LLM
    for model, flight_plan in all_flight_plans.items():
        print(f"Flight plan generated by {model}:")
        for section_name, section_content in flight_plan.items():
            print(f"{section_name}:\n{section_content}\n")

Response for Communication Plan was truncated or incomplete. Retrying with temperature 0.7...
Flight plan generated by gpt-4:
Launch Date and Time:
After analyzing the data, the optimal launch date and time for the Orion spacecraft mission to the Moon, considering minimum radiation exposure and optimal lunar phases, is:

**2026-11-17 at 02:43:00 UTC**.

Please note that this date assumes no major changes to the forecasted solar activity, Earth-Moon alignment, and other celestial events. It is recommended to regularly update and adjust the flight plan as the launch date approaches to account for any changes in these variables.

Trajectory:

**Launch:**
The launch of a spacecraft generally involves several stages, from preparation to liftoff to reaching a parking orbit. Here is a typical process:

1. Launch Vehicle: The launch vehicle, often a rocket, is the primary means of transporting the spacecraft from Earth into space. The launch vehicle is usually composed of several stages, each 